In [14]:
# Importing libraries
import tensorflow as tf
print(tf.__version__)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

2.0.0-alpha0


In [2]:
dt = pd.read_csv('/Users/ruhidmirzayev/Desktop/ ML_Projects/JobEmails_csv.csv')
print(dt.shape)

(137, 2)


/bin/sh: pip: command not found


In [3]:
dt.head(20)

,Emails,NextStep
0,Thank you for your interest in Bonsai!\nWe rec...,No
1,Thank you for applying to the Business Coordin...,No
2,Thank you for your interest in Morningstar and...,No
3,Thank you for applying to the Investment Assoc...,No
4,Thank you for applying for the role of Persona...,No
5,Thank you for taking the time to consider HSBC...,No
6,"Unfortunately, Numeris has decided not to move...",No
7,"Unfortunately, North Temp Glass Ltd. has decid...",No
8,"Unfortunately, Prodigy Systems Inc. has decide...",No
9,"Unfortunately, Amax Leather Ltd has decided no...",No


In [16]:
from wordcloud import WordCloud,STOPWORDS
plt.figure(figsize = (20,20)) # Positive Review Text
wc = WordCloud(max_words = 2000 , width = 1600 , height = 800).generate(" ".join(df[df.NextStep == 'Yes'].review))
plt.imshow(wc , interpolation = 'bilinear')

ModuleNotFoundError: No module named 'wordcloud'

In [4]:
dt['NextStep'].value_counts()

No     90
Yes    47
Name: NextStep, dtype: int64

In [5]:
train_emails = np.array(dt.Emails[:100])
train_NextStep = np.array(dt.NextStep[:100])

test_emails = np.array(dt.Emails[100:])
test_NextStep = np.array(dt.NextStep[100:])

print(train_emails.shape, train_NextStep.shape)
print(test_emails.shape, test_NextStep.shape)

(100,) (100,)
(37,) (37,)


In [7]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"


from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_emails)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(train_emails)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

test_emails = tokenizer.texts_to_sequences(test_emails)
testing_padded = pad_sequences(test_emails,maxlen=max_length)

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [9]:
num_epochs = 10
model.fit(padded, test_emails, epochs=num_epochs, validation_data=(testing_padded, test_emails))

ValueError: Error when checking model target: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 array(s), but instead got the following list of 37 arrays: [array([[333],
       [  1],
       [  1],
       [336],
       [148],
       [102],
       [  2],
       [  1],
       [271],
       [ 55],
       [126],
       [  2],
       [ 29],
       [ 21],
   ...